In [ ]:
!pip install nltk PyPDF2 transformers torch


In [5]:
import os
import re
import nltk
import PyPDF2
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # Required for WordNet
nltk.download('punkt')    # For tokenization
nltk.download('stopwords')  # For stopwords


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
# Load NLP Model (T5 for text generation)
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Predefined stopwords
sw = set(stopwords.words('english'))

def extract_text_from_pdf(pdf_path):
    """Extract text from a local PDF file."""
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() + " "
    return text.strip()

def clean_text(text):
    """Clean text by removing special characters and stopwords."""
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    words = word_tokenize(text)  # Tokenize words
    clean_words = [word for word in words if word not in sw]  # Remove stopwords
    return ' '.join(clean_words)


def generate_text(cleaned_text):
    """Generate new text using T5 NLP model."""
    input_text = "summarize: " + cleaned_text  # T5 works well with "summarize:" prefix
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=150, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [19]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

# Set stopwords
sw = set(stopwords.words('english'))

def clean_text(text):
    """Clean text by removing special characters and stopwords."""
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    words = word_tokenize(text)  # Tokenize words
    clean_words = [word for word in words if word not in sw]  # Remove stopwords
    return ' '.join(clean_words)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
pdf_path = "/kaggle/input/sample2/Tender summarization sumarization common festurees.pdf"  # Update this with the correct file path

# Step 1: Extract Text
raw_text = extract_text_from_pdf(pdf_path)
print("Extracted Text:", raw_text[:4000])  # Print first 500 characters

# Step 2: Clean Text
cleaned_text = clean_text(raw_text)
print("Cleaned Text:", cleaned_text[:4000])  # Print first 500 characters

# Step 3: Generate Summary/Text
generated_text = generate_text(cleaned_text)
print("Generated Text:", generated_text)
